#### Installation

The pywapor package depends on several other packages, most of them get installed automatically when we install pywapor. The GDAL package needs to be installed manually however. Luckily, it is already installed on the backend computer used by Google Colab. We can verify that GDAL is installed by running the following commands.

In [ ]:
from osgeo import gdal
print("Using gdal version", gdal.__version__)

Now that we know that Python is able to import the GDAL package, we can install pywapor by running the following command.



In [ ]:
!pip install pywapor --quiet

If everything went well, we can now import pywapor in Python, let's try it (fingers crossed)!

In [ ]:
import pywapor
print("Using pywapor version:", pywapor.__version__)

#### ETLook Input

In order to run the ETLook model, we first need to collect the necessary inputs. For your convenience, the pywapor package has a function that can collect all this data from selected sources and make sure the data is stored in the correct format. In one of the following lessons we will look more closely at how you can adjust these inputs. Here we will focus on running a basic configuration.

Because some of the data portals used require a user to login with a username and a password, we first need to set those up. Most importantly, we will need a `NASA Earthdata Login` to be able to collect the MODIS and MERRA2 datasets, which can be created [over here](https://urs.earthdata.nasa.gov/users/new).

> **Note**
>
> After creating your account, you still need to accept some 'Terms of Use', before you can continue with this notebook. To do that, login to your newly created account and go to
>
> `Applications > Authorized Apps > Approve More Applications`
>
> There, make sure the two following applications are authorized:
> 1. `NASA GESDISC DATA ARCHIVE`
> 2. `LP DAAC OPeNDAP`

Once you have your account set up, we can enter our username and password by running the following code (if you skip this step, you'll be prompted for your username and password once the script starts downloading any product that requires them).


In [ ]:
pywapor.collect.accounts.setup("NASA")

Now that we have our account set up we can start defining our period and area-of-interest (AOI). 

First we define a time period through `timelim`, our boundingbox through `latlim` and `lonlim` and we give a `project_folder` in which all our data will be stored. Here I've chosen an period of exactly 10 days which is the default composite length of pyWAPOR. We'll talk more about composites in the next lesson.

Note that for `latlim` the first value refers to the southern border of your AOI, so this value should always be smaller than the second. For `lonlim`, the first value refers to the western border of your AOI.

In [ ]:
folder = r"/Users/hmcoerver/Local/1_level1"
timelim = ["2021-08-11", "2021-09-21"]
lonlim = [68.3, 69.5]
latlim = [25.3, 26.3]

> **Question**
> 
> Which area is covered by these longitude and latitude coordinates?

Then we can run `pywapor.pre_et_look.main()` to start preparing the inputs for the model. This will take a bit of time (around 15min).

In [ ]:
ds  = pywapor.pre_et_look.main(folder, latlim, lonlim, timelim)

> **Question**
> 
> You can see that the function prints all kind of information. Can you list all the variables that have been downloaded (or "collected")?

> **PRE_ET_LOOK Variables**
> 
> * NDVI
> * etc..
> * 
> * 

Once you see the output line `< PRE_ET_LOOK`, the code has finished. The function has created a bunch of files in your `project_folder` and returned a variables `ds`. This variable contains a `xarray.Dataset`. XArray is an Python-package that let's you work with large multi-dimensional datasets. We can see the file this dataset is stored in like this.

In [ ]:
fh = ds.encoding["source"]
print(fh)

> **Question**
> 
> How large (in MBs) is this file?

As you can see, the dataset is stored inside a file called `et_look_in.nc`, this file contains all the data we need to run the `et_look` model!

We can have a closer look at the contents of the datasets by simply calling `ds` like this.

In [ ]:
ds

The output of calling `ds` is interactive, you can click on different parts of the output to get more informationa and expand some parts.

> **Question**
> 
> * How many dimensions does this dataset have?
> 
> * How many variables does the dataset contain?
> 
> * Does every variable have the same amount of dimensions?
>
> * How many pixels does the variable `ndvi` contain in total?

Just like geotiff files, this dataset also contains information like it's coordinate reference system and boundaries. We can access that information like this.

In [ ]:
print(ds.rio.crs)
print(ds.rio.bounds())
print(ds.rio.resolution())

> **Question**
> 
> * What is the resolution of this dataset (don't forget the units)?
> 
> * Given the bounds and the resolution, how many pixels does the dataset contain? Does that match with what you found in the previous question?

XArray datasets are also easy to plot, for example, we can create a map of `z` like this (Note that XArray is smart enough to automatically fill in the units and the description of the variable!).

In [ ]:
ds.z.plot()

If we want to make a map of a variable with more than 2 dimensions, we'll first have to select for which time (i.e. `time_bins`) we want to create the map. Here I'm selecting the first map, i.e. at index 0.

In [ ]:
ds.r0.isel(time_bins = 0).plot()

> **Question**
> 
> * Which time does the index 0 refer to?
> 
> * Can you make a plot of the `ndvi` for another date/time?

In [ ]:
# Create another plot here.

If for some reason you have lost the variable `ds` (maybe you restarted you computer) and don't want to run `pre_et_look` again, you can simply open the file specified by `fh` like this. (The `decode_coords` keyword is used to make sure the CRS information is loaded correctly.)

In [ ]:
ds = pywapor.general.processing_functions.open_ds(fh)

#### Running ETLook

Now that we have created the input file for the model, we can run the model! We simply pass the `ds` to `pywapor.et_look.main()`.

In [ ]:
ds_out = pywapor.et_look.main(ds)

This should go pretty quickly. Again, we can see a new netCDF file has been created. Do you remember how to print the path to the file?

> **Question**
> 
> Run the command to print the paths to the file.
> 
> How large is this file?

In [ ]:
# Find the path to the file linked to the new dataset here.

We can check the contents of the new dataset just like we did before.

In [ ]:
ds_out

> **Question**
> 
> How many variables are present in the output?
> 
> Can you find the resolution of this output dataset?

In [ ]:
# Write code to find the resolution of the output dataset here.

> **Question**
> 
> Can you create a map of the ET?

In [ ]:
# Write some code to create a plot of the ET here.

Besides the variables exported in the output dataset, the model calculates many more (intermediate) variables. You can export all variables by specifying a keyword argument called `export_vars` set to `"all"` when you can run ET_Look.

> **Question**
> 
> Can you run ET_Look and export all variables?

In [ ]:
ds_out_all = pywapor.et_look.main(fh, # ... finish the code to export all variables here.

> **Question**
> 
> * How many variables does the output contain now?
> 
> * What is the filesize of the output?

In [ ]:
# Inspect the dataset here.

#### Now its your turn

> **Question**
> 
> Can you run `pre_et_look` for another region and/or another year ([bboxfinder](http://bboxfinder.com) can help you choose a region).

In [ ]:
# Define a project folder, time period and area-of-interest here.
folder = r""
timelim = ["", ""]
lonlim = []
latlim = []

In [ ]:
# Run pre_et_look here.

In [ ]:
# Run et_look here.

> **Question**
> 
> * Does the variable `et_24_mm` contains any holes with missing data?
> 
> * If so, can you figure out what is causing these holes (hint: check the input data)?
> 
> * Do you have any ideas on how to fill these gaps of missing data?

> **Question**
> 
> Can you think of any ways to validate if our outputs are correct?

> **ET Validation methods**
> 
> 1. ...
> 2. ...
> 3. ...
> 4. etc.